In [1]:
from qiskit import Aer
from qiskit.algorithms import VQE, QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit.utils import QuantumInstance
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider, YahooDataProvider
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.applications import OptimizationApplication
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.utils import algorithm_globals

import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas
import pandas as pd

def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

def print_result(result):
    selection = result.x
    value = result.fval
    print('Optimal: selection {}, value {:.4f}'.format(selection, value))
    
    eigenstate = result.min_eigen_solver_result.eigenstate
    eigenvector = eigenstate if isinstance(eigenstate, np.ndarray) else eigenstate.to_matrix()
    probabilities = np.abs(eigenvector)**2
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, num_assets)
        value = QuadraticProgramToQubo().convert(qp).objective.evaluate(x)
        value = portfolio.to_quadratic_program().objective.evaluate(x)
        probability = probabilities[i]
        print('%10s\t%.4f\t\t%.4f' %(x, value, probability))

In [2]:
#global random seed used throuhgout
seed = 123

In [3]:
# Generate expected return and covariance matrix from (random) time-series

#Todo pick candidate stocks, 3-8 are recommended, stick to 4 if you're going to submit a real quantum experiment to IBM's cloud:

stocks = ['IBM','GOOGL','AMZN','ACN','NVDA','MSFT']
    
num_assets = len(stocks)

#IBM has done us a favour and provided a cute data provider they designed.
data = YahooDataProvider(tickers=stocks,
                 start=pandas.Timestamp('2020-01-01T12'),
                 end=pandas.Timestamp.now())
data.run()

# Here we pull apart the IBM qiskit_finance.data_provider internals to retreive our data and format as a pandas DataFrame
df = pandas.DataFrame({data._tickers[tidx]:data._data[tidx] for tidx in range(len(data._tickers))})

# Provided by IBM
mu = data.get_period_return_mean_vector()
sigma = data.get_period_return_covariance_matrix()

In [4]:
q = 0.50                   #  risk appetite 
budget = num_assets  //2   #  stocks to allocate
penalty = num_assets       #  set parameter to scale the budget penalty term
bounds = None              #  Allocation percent: None: 100%

portfolio = PortfolioOptimization(expected_returns=mu, covariances=sigma, risk_factor=q, budget=budget,bounds=bounds)
qp = portfolio.to_quadratic_program()
qp

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Portfolio optimization

Minimize
 obj: - 0.000227142144 x_0 - 0.001725374059 x_1 - 0.001483546505 x_2
      - 0.001370686047 x_3 - 0.004049065175 x_4 - 0.001777396854 x_5 + [
      0.000451047977 x_0^2 + 0.000445697433 x_0*x_1 + 0.000270044511 x_0*x_2
      + 0.000580730945 x_0*x_3 + 0.000527422191 x_0*x_4 + 0.000481323993 x_0*x_5
      + 0.000422553998 x_1^2 + 0.000545766765 x_1*x_2 + 0.000586743492 x_1*x_3
      + 0.000889924639 x_1*x_4 + 0.000739965795 x_1*x_5 + 0.000419346349 x_2^2
      + 0.000394812819 x_2*x_3 + 0.000824179204 x_2*x_4 + 0.000625949569 x_2*x_5
      + 0.000423195849 x_3^2 + 0.000772451240 x_3*x_4 + 0.000653350074 x_3*x_5
      + 0.001042006698 x_4^2 + 0.001077048263 x_4*x_5 + 0.000478800713 x_5^2 ]/2
Subject To
 c0: x_0 + x_1 + x_2 + x_3 + x_4 + x_5 = 3

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4 x_5
En

In [5]:
algorithm_globals.random_seed = seed+1
backend = Aer.get_backend('statevector_simulator')


cobyla = COBYLA()
cobyla.set_options(maxiter=500)

ry = TwoLocal(num_assets, 'ry', 'cz', reps=3, entanglement='full')

quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)

vqe_mes = VQE(ry, optimizer=cobyla, quantum_instance=quantum_instance)
vqe = MinimumEigenOptimizer(vqe_mes)

result = vqe.solve(qp)


# print(result.fval)
print_result(result)

/opt/conda/lib/python3.8/site-packages/sympy/core/expr.py:3949: SymPyDeprecationWarning: 

expr_free_symbols method has been deprecated since SymPy 1.9. See
https://github.com/sympy/sympy/issues/21494 for more info.

  SymPyDeprecationWarning(feature="expr_free_symbols method",


Optimal: selection [0. 1. 0. 0. 1. 1.], value -0.0052

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[0 0 1 1 0 1]	-0.0031		0.5087
[0 1 0 1 0 1]	-0.0032		0.3223
[0 0 0 1 1 1]	-0.0050		0.1447
[0 0 1 1 1 0]	-0.0050		0.0188
[0 0 1 0 1 1]	-0.0051		0.0022
[0 1 1 1 0 0]	-0.0032		0.0011
[0 1 1 0 0 1]	-0.0034		0.0006
[1 0 1 1 0 0]	-0.0018		0.0003
[0 1 0 1 1 0]	-0.0051		0.0003
[0 0 1 0 1 0]	-0.0044		0.0002
[0 1 0 0 1 1]	-0.0052		0.0001
[0 0 0 0 1 1]	-0.0045		0.0001
[0 1 1 1 1 0]	-0.0055		0.0001
[1 0 0 1 1 1]	-0.0042		0.0001
[1 1 0 1 0 0]	-0.0019		0.0001
[0 1 0 0 0 0]	-0.0015		0.0001
[0 1 1 0 0 0]	-0.0025		0.0000
[1 0 1 1 1 0]	-0.0043		0.0000
[0 1 1 0 1 1]	-0.0055		0.0000
[0 1 0 1 0 0]	-0.0024		0.0000
[0 0 1 1 0 0]	-0.0022		0.0000
[0 0 0 0 1 0]	-0.0035		0.0000
[1 1 1 1 1 0]	-0.0046		0.0000
[1 0 0 1 0 1]	-0.0018		0.0000
[0 1 0 1 1 1]	-0.0054		0.0000
[0 0 1 1 1 1]	-0.0053		0.0000
[0 0 1 0 0 1]	-0.0025		0.0000
[0

In [7]:
selected_stocks = list(np.array(stocks)[result.x.astype('bool')])
print(f"Stocks from our Qunatum Stock Picking algorthm selected: {selected_stocks}")


Stocks from our Qunatum Stock Picking algorthm selected: ['GOOGL', 'NVDA', 'MSFT']


In [ ]:
sel = df[selected_stocks]
all = df
returns = sel.pct_change().dropna().mean(axis=1)
returns_all = all.pct_change().dropna().mean(axis=1)
m_all = 10000 * (returns_all+1).cumprod()
m_sel = 10000 * (returns+1).cumprod()
m_sel.plot()
m_all.plot()
plt.legend(['Selection','all'])
print(f"IR Selection: {(returns.mean()/returns.std())*np.sqrt(252)}")
print(f"IR All: {(returns_all.mean()/returns_all.std())*np.sqrt(252)} \n")
# print(f"CAGR selection: {round(((1+(m_sel.iloc[-1]/10000))**(1/len(df)/252)}%")
# print(f"CAGR all: {round((1+(m_all.iloc[-1]/10000))**(1/len(df)/252)),4)}%")
print(f"CAGR Selection: {round(100*((m_all.iloc[-1]/10000)-1)**(1/(len(df)/252)),2)}%")
